A few things you should keep in mind when working on assignments:

1. Make sure you fill in any place that says `YOUR CODE HERE`. Do **not** write your answer in anywhere else other than where it says `YOUR CODE HERE`. Anything you write anywhere else will be removed or overwritten by the autograder.

2. Before you submit your assignment, make sure everything runs as expected. Go to menubar, select _Kernel_, and restart the kernel and run all cells (_Restart & Run all_).

3. Do not change the title (i.e. file name) of this notebook.

4. Make sure that you save your work (in the menubar, select _File_ → _Save and CheckPoint_)

5. You are allowed to submit an assignment multiple times, but only the most recent submission will be graded.

# Problem 4. Decision Trees.

In this problem, we fit a decision tree classifier that takes the day of the week and depature delays as input and predicts whether a flight is on time or not.

In [ ]:
import numpy as np
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split, KFold
from sklearn.metrics import accuracy_score

from nose.tools import assert_is_instance, assert_equal, assert_almost_equal
from numpy.testing import assert_array_equal, assert_array_almost_equal
from pandas.util.testing import assert_index_equal

We use the same [airline on-time performance data](http://stat-computing.org/dataexpo/2009/) from the lessons. You can find the descriptions [here](http://stat-computing.org/dataexpo/2009/). We use five columns: `Month`, `DayOfWeek`, `ArrDelay` `DepDelay`, and `Origin`. (Note: we are using one more column than we did in problems 2 and 3.)

In [ ]:
filename = "/home/data_scientist/data/2001.csv"
usecols = (1, 3, 14, 15, 17)
names = ["Month", "DayOfWeek", "ArrDelay", "DepDelay", "Origin"]

all_data = pd.read_csv(filename, header=0, na_values=["NA"], usecols=usecols, names=names)

We perform some data pre-processing, similarly to the [Introduction to Logistic Regression](https://github.com/UI-DataScience/accy571-fa16/blob/master/Week6/notebooks/intro2lr.ipynb) notebook.

To simplify the computations, we first extract only those flights that depart from Willard airport (CMI). After this, we drop all rows that have missing values ("`NA`") in any of the columns.

We next create a categorical column, _arrival late_, that is zero if the flight arrived less than 5 minutes after the scheduled arrival time, or one if it arrived more than this number of minutes after the scheduled time. We will use this
to train our logistic regressor.

To save memory, we drop the columns that we no longer need: the origin airport and arrival delay columns.

Finally, we reset the index.

In [ ]:
local = all_data[all_data["Origin"] == "CMI"].dropna()

local["ArrLate"] = (local["ArrDelay"] > 5).astype(int)

local = local.drop(["Origin", "ArrDelay"], axis=1)

local = local.reset_index(drop=True)

Let's print out the first 10 columns of the resulting data frame.

```python
>>> print(local.head(10))
```
```
   Month  DayOfWeek  DepDelay  ArrLate
0      1          1      15.0        1
1      1          2      -5.0        1
2      1          3      52.0        1
3      1          4      12.0        0
4      1          5       0.0        0
5      1          7     152.0        1
6      1          1      51.0        1
7      1          2       3.0        0
8      1          3      -7.0        0
9      1          4      14.0        0
```

In [ ]:
print(local.head(10))

In the previous problems, we used a validation set to adjust the hyperparameters. The <a href="https://en.wikipedia.org/wiki/Cross-validation_(statistics)#k-fold_cross-validation">k-fold cross-validation</a> technique extends the idea of validation set. You can read more about $k$-fold CV on <a href="https://en.wikipedia.org/wiki/Cross-validation_(statistics)#k-fold_cross-validation">Wikipedia</a> or [scikit-learn docs](http://scikit-learn.org/stable/modules/cross_validation.html).

We are going to use $k$-fold CV to

1. measure the performance increase with increasing number of trees, and
2. optimize one of the hyperparameters, `max_features`.
We split `local` into a training set (used for training our model), a validation set (used for determining hyperparameters), and a test set (used for evaluating our model's performance).

In the following code cell, we first split `local` into 80:20 training and test sets, and then use [sklearn.cross_validation.KFold()](http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.KFold.html) to get a $k$-fold cross-validation iterator with `n_fold=4` (4-fold cross-validation).

In [ ]:
# Evaluate the model by splitting into train and test sets
X = local.drop("ArrLate", axis=1)
y = np.ravel(local["ArrLate"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)
kf = KFold(len(X_train), n_folds=4, random_state=0)

## Train a decision tree model

Now that we have standardized the test sets, we are ready to apply the decision tree classifier.


- Write a function named `fit_dt_and_predict()` that fits a decision forest classifier using [DecisionTreeClassifier()](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) in scikit learn.

- Note that the function takes six arguments. **You must use `max_features` and `random_state`**, but it is not necessary that you use all of the other four arguments (`X_train`, `X_test`, `y_train`, and `y_test`). You should decide which arguments are needed and which are not.

- If you read the [sklearn.tree.DecisionTreeClassifer documentation](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html), there are many optional parameters that you can use in `DecisionTreeClassifier()`, e.g., `criterion`, `splitter`, etc. We will use the `max_features` and `random_state` parameters in this notebook. Use the `max_features` variable in the function definition in `DecisionTreeClassifier()`. Use defaults values for all optional parameters except `max_features` and `random_state`.

- `max_features` is the number of features that are considered for each split. In my experience, this is the most important parameter. The rule of thumb that gets thrown around a lot is the square root of total number of features, but it's best to fine tune this parameter.

- Without the `random_state` parameter, the algorithm has a random element. If you provide an integer to the `random_state` paramter, the algorithm becomes determinitstic and reproducible. So, for example, your code will look something like
```python
def fit_dt_and_predict(X_train, X_test, y_train, y_test, max_features, random_state):
    # YOUR CODE HERE
    DecisionTreeClassifier(
        # YOUR CODE HERE
        ,random_state=random_state
    )
    # YOUR CODE HERE
```

In [ ]:
def fit_dt_and_predict(X_train, X_test, y_train, y_test, max_features, random_state):
    """
    Fits a SVM classifier on the training set.
    Returns the predicted values on the test set.
    
    Paramters
    ---------
    X_train: A pandas data frame. The features of the training set.
    X_test: A pandas data frame. The features of the test set.
    y_train: A numpy array. The labels of the training set.
    y_test: A numpy array. The labels of the test set.
    max_features: An int. The number of features to consider when looking for the best split.
    random_state: An int. The seed used by the random number generator.
    
    Returns
    -------
    A 1-D numpy array. 
    """
    
    # YOUR CODE HERE
    
    return result

In [ ]:
for max_features in [1, 2, 3]:
    scores = []
    for train, valid in kf:
        y_pred = fit_dt_and_predict(
            X_train.iloc[train], X_train.iloc[valid], y_train[train], y_train[valid],
            max_features=max_features, random_state=0
        )
        scores.append(accuracy_score(y_train[valid], y_pred))
    print("When max_features is {0}, the mean accuracy score is {1:3.1f} %.".format(max_features, 100.0 * np.mean(scores)))

Using only 1 feature when looking for the best split yields the greatest accuracy, so we choose `max_features=1` as the optimal model for our data set. Now that we have decided on our model, we can now train on the entire training set, and then use the test set to evaulate the performance.

In [ ]:
y_pred_final = fit_dt_and_predict(
    X_train, X_test, y_train, y_test,
    max_features=1, random_state=0
)
accuracy_final = accuracy_score(y_test, y_pred_final)
print("test set accuracy = {0:3.1f} %".format(100.0 * accuracy_final))

In [ ]:
assert_is_instance(y_pred_final, np.ndarray)
assert_equal(len(y_pred_final), len(y_test))
assert_array_equal(
    np.where(y_pred_final != y_test)[0],
    [  5,   6,  12,  24,  26,  29,  31,  38,  41,  46,  49,  61,  64,
       67,  68,  72,  83,  98, 110, 111, 125, 128, 131, 141, 142, 149,
       156, 162, 167, 168, 186, 196, 206, 208, 213, 216, 219, 222, 228,
       229, 236, 242, 250, 251, 252, 257, 261, 280, 295, 297, 304, 308,
       317, 322, 327]
)
assert_almost_equal(accuracy_final, 0.83775811209439532)